### PyTorch implementation of Random Audio Style Transfer

based on:
https://pytorch.org/tutorials/advanced/neural_style_tutorial.html

In [ ]:
import librosa
import torch

from IPython.display import display, Audio

from utils import plot_spectrum, read_audio_spectrum, spectrum_to_audio
from model import RandomCNN, run_style_transfer
from model import get_input_optimizer, get_style_model_and_losses

In [ ]:
CONTENT_PATH = "wavs/songs/imperial.mp3"
STYLE_PATH = "wavs/songs/usa.mp3"

content_s, content_sr, content_p = read_audio_spectrum(CONTENT_PATH)
style_s, style_sr, style_p = read_audio_spectrum(STYLE_PATH)

In [ ]:
plot_spectrum(content_s)
content_wav = spectrum_to_audio(content_s)
display(Audio(content_wav, rate=content_sr))

plot_spectrum(style_s)
style_wav = spectrum_to_audio(style_s)
display(Audio(style_wav, rate=style_sr))

In [ ]:
mcnn = RandomCNN()

content = torch.from_numpy(content_s)[None, None, :, :]
style = torch.from_numpy(style_s)[None, None, :, :]
result = torch.randn(content.data.size())

result_s = (content_s + style_s) / 2
result = torch.from_numpy(result_s)[None, None, :, :]

In [ ]:
model, style_losses, content_losses = get_style_model_and_losses(mcnn, style, content)

result.requires_grad_(True)
model.eval()
model.requires_grad_(False)

optimizer = get_input_optimizer(result)

optimizer.zero_grad()
model(result)
style_score = 0
content_score = 0

for sl in style_losses:
    style_score += sl.loss
for cl in content_losses:
    content_score += cl.loss

style_score *= 1e6
content_score *= 1

loss = style_score + content_score
loss.backward()

In [ ]:
result_s = result.cpu().data.numpy().squeeze()
plot_spectrum(result_s)

In [ ]:
result_wav = spectrum_to_audio(result_s, p=content_p, rounds=50)
display(Audio(result_wav, rate=style_sr))